<a href="https://colab.research.google.com/github/ParkerStephenJohnson/OnMart/blob/main/Copy_of_Phase_5_Surpirse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

import os
import csv
import surprise
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import SVDpp
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from tabulate import tabulate

np.random.seed(1234)

import numpy as np

In [ ]:
transactions = pd.read_csv("transactions_preprocessed.csv")

In [ ]:
transactionsdf = pd.DataFrame()
transactionsdf['CustomerID'] = transactions['CustomerID'].astype(str)
transactionsdf['Product_Name'] = transactions['Product_Name'].astype(str)
transactionsdf['Review_Rating'] = transactions['Review_Rating'].astype(int)
transactionsdf = transactionsdf.dropna()

In [ ]:
# Number of unique customers
len(transactionsdf['CustomerID'].unique())

1057

In [ ]:
transactionsdf.to_csv('transaction_train.csv', index=False)

with open("transaction_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open("transaction_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)

In [ ]:
file_path = 'transaction_test.csv'

# Create reader for custom dataset
reader = Reader(line_format='user item rating', sep=',')

def get_top_n(predictions, n=3):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
%timeit algo.fit(trainset)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n_SVD = get_top_n(predictions, n=3)

10 loops, best of 5: 138 ms per loop


In [ ]:
# Print the recommended items for each user
for uid, user_ratings in top_n_SVD.items():
    print(uid, [iid for (iid, _) in user_ratings])
    


254-72-5938 ['Redmi']
856-88-9899 ['Redmi', 'JBL']
375-90-5301 ['Redmi', 'Bose', 'JBL']
704-41-5593 ['Samsung s10']
033-16-4037 ['Redmi', 'Bose']
404-86-6657 ['Samsung s10', 'JBL']
269-29-9833 ['Redmi', 'Bose']
032-36-3151 ['Redmi', 'Bose', 'JBL']
538-94-1269 ['Redmi', 'Bose', 'JBL']
841-77-1794 ['Samsung s10', 'Bose']
103-65-2527 ['Samsung s10', 'Bose', 'JBL']
745-59-8085 ['Bose', 'Samsung s10']
346-12-7799 ['Bose']
471-57-6708 ['Samsung s10', 'Bose']
366-42-7104 ['Redmi', 'Bose', 'JBL']
764-44-6294 ['Bose', 'JBL']
234-62-1733 ['Bose', 'Samsung s10', 'JBL']
848-09-1487 ['Bose', 'JBL']
186-01-5765 ['Samsung s10', 'Bose', 'JBL']
762-70-4838 ['Bose', 'JBL']
605-32-0992 ['Redmi', 'JBL']
894-97-3580 ['Bose', 'JBL']
807-44-0802 ['Bose', 'Samsung s10', 'JBL']
576-08-0975 ['Bose', 'JBL']
521-76-2086 ['Redmi', 'Bose', 'JBL']
472-28-7031 ['Samsung s10', 'Bose']
219-65-7672 ['Bose', 'JBL']
078-06-1635 ['Redmi', 'Bose', 'JBL']
472-86-9802 ['Redmi', 'Samsung s10', 'JBL']
683-81-0851 ['Redmi', 'Sam

In [ ]:
# top n match unique customer?
len(top_n_SVD)

1041

In [ ]:
out = open('SVD_Recommendations.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n_SVD.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


In [ ]:
SVD_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0364  0.0347  0.0377  0.0358  0.0352  0.0359  0.0011  
MAE (testset)     0.0286  0.0274  0.0290  0.0282  0.0271  0.0281  0.0007  
Fit time          0.55    0.23    0.13    0.12    0.13    0.23    0.17    
Test time         0.02    0.01    0.02    0.01    0.02    0.01    0.00    


In [ ]:
# First train an SVD++ algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVDpp()
%timeit algo.fit(trainset)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n_SVDpp = get_top_n(predictions, n=3)

1 loop, best of 5: 287 ms per loop


In [ ]:
SVDpp_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0756  0.0826  0.0887  0.0831  0.0761  0.0812  0.0049  
MAE (testset)     0.0479  0.0537  0.0578  0.0534  0.0491  0.0524  0.0035  
Fit time          0.37    0.49    0.39    0.39    0.47    0.42    0.05    
Test time         0.01    0.01    0.01    0.02    0.01    0.01    0.00    


In [ ]:
def compute_recommendations_experiment_SVD(CustomerID):
    """
    Create a function given the customerID predicts the score for each product to recommend.
    :param customerID: String containing the customerID
    :return: prints recommendations for the user with customer = customerID with their scores
    """
    recommendation = (pd.DataFrame.from_dict(top_n_SVD.items())).reset_index()
    recommendation.columns = ['Count','CustomerID', 'Recommendation_list']
    get_recommendations = (recommendation[recommendation['CustomerID'] == CustomerID]['Recommendation_list'].values)[0]
    return print('\n\t   CustomerID :',CustomerID,'\n\n' + 
                          tabulate(get_recommendations,
                          headers=["Product Name","Recommendation Score"],
                          tablefmt='fancy_grid'))

def compute_recommendations_experiment_SVDpp(CustomerID):
    """
    Create a function given the customerID predicts the score for each product to recommend.
    :param customerID: String containing the customerID
    :return: prints recommendations for the user with customer = customerID with their scores
    """
    recommendation = (pd.DataFrame.from_dict(top_n_SVDpp.items())).reset_index()
    recommendation.columns = ['Count','CustomerID', 'Recommendation_list']
    get_recommendations = (recommendation[recommendation['CustomerID'] == CustomerID]['Recommendation_list'].values)[0]
    return print('\n\t   CustomerID :',CustomerID,'\n\n' + 
                          tabulate(get_recommendations,
                          headers=["Product Name","Recommendation Score"],
                          tablefmt='fancy_grid'))

In [ ]:
compute_recommendations_experiment_SVD('127-37-8482')


	   CustomerID : 127-37-8482 

╒════════════════╤════════════════════════╕
│ Product Name   │   Recommendation Score │
╞════════════════╪════════════════════════╡
│ Samsung s10    │                2.04013 │
├────────────────┼────────────────────────┤
│ Bose           │                2.01633 │
├────────────────┼────────────────────────┤
│ JBL            │                1.02141 │
╘════════════════╧════════════════════════╛


In [ ]:
compute_recommendations_experiment_SVDpp('127-37-8482')


	   CustomerID : 127-37-8482 

╒════════════════╤════════════════════════╕
│ Product Name   │   Recommendation Score │
╞════════════════╪════════════════════════╡
│ Bose           │                2.00394 │
├────────────────┼────────────────────────┤
│ Samsung s10    │                1.99002 │
├────────────────┼────────────────────────┤
│ JBL            │                1.0132  │
╘════════════════╧════════════════════════╛
